In [1]:
import openai
import pandas as pd
from tqdm import tqdm
import json
import time
import random
import pickle
import math

#load the API
openai.api_type = "azure"
openai.api_version = "2023-07-01-preview"
openai.api_base = "https://structuralfactorsofsuicide.openai.azure.com/"
openai.api_key = "f678e6405d5f46099018cd5c9370758f"

FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"
FinancialPredicted = "Financial - Predicted"
LegalPredicted = "Legal - Predicted"
SchoolPredicted = "School - Predicted"
HealthPredicted = "Health - Predicted"

In [94]:
ZeroShotPrompt = "Suicide is the act of intentionally causing one's own death or taking their own life by any means. You are a public health professional. You are going through news articles from across the USA that mention suicide and your task is to identify the cause of the suicide talked about in the article. The article may not use the word ‘suicide’, instead explain it in another way, such as ‘he took his own life’ or a different expression that has the same meaning. \n You have to categorize the articles in one or more of the following categories: \n\n 1) 'Financial/Job Problem': If there is mention of a person or multiple people who killed themselves due to financial or work related problems or bad financial state or an incident at their place of employment. \n 2) ‘School Problem': If there is mention of a person or multiple people who killed themselves due to an academic or school related problem. \n 3) 'Health Problem': If there is mention of a person or multiple people who killed themselves due to lack of access to mental or physical health care or treatment. \n 4) ‘Legal Problem': If there is mention of a person or multiple people who killed themselves after facing a legal problem or being involved or suspected or targeted in a violation of the law. \n 5) ‘None’: If there is no explicit mention of a person who committed suicide or killed themselves or if suicide was attempted or threatened but the person did not die or if the reason of suicide is not aligned with any of the above 4 categories. \n\n Now, read the following news article answer the following 2 questions: \n 1. Is there mention of a person or multiple people who died after committing suicide or killed themselves? Provide your answer in the following format: \n Answer 1: Yes or No  \n 2. Identify which of the above categories does the article belong to. An article can belong to more than one category. Also provide a reasoning for the same in the following format:\n Answer 2: \n Reasoning: \n\n Article: "

In [95]:
combinedfewshot = "Suicide is the act of intentionally causing one's own death or taking their own life by any means. You are a public health professional. You are going through news articles from across the USA that mention suicide and your task is to identify whether financial struggles or problems related to their job/profession or an incident at their place of employment influenced or led to the suicide talked about in the article. The article may not use the word ‘suicide’, instead explain it in another way, such as ‘he took his own life’ or a different expression that has the same meaning. Suicide in news reports can be expressed in various terms, such as mentioning the act of suicide, or mentioning that someone killed themselves, or someone ended their own life or someone took their own life and other similar expressions. \n You have to categorize the articles in one or more of the following categories: \n\n 1) 'Financial/Job Problem' : If there is mention of a person or multiple people who killed themselves primarily due to financial or work related struggles or an incident at their place of employment. \n 2) ‘School Problem' : If there is mention of a person or multiple people who killed themselves primarily due to an incident at school or an academic/school problem experienced by them. \n 3) 'Health Problem' : if there is mention of a person or multiple people who killed themselves primarily due to lack of access to mental or physical health care or treatment. \n 4) ‘Legal Problem' : If there is mention of a person or multiple people who killed themselves primarily after facing a law related problem or being involved or suspected in a violation of the law by the authorities. \n 5) ‘None’ : If there is no mention of a person committing suicide or if the reason of suicide is not aligned with any of the above 4 categories. \n\n Following are the examples of news articles talking about suicide and the answer to which category it falls into along with the reasoning behind that answer:\n\n Article: a suicide in texas there is a strange fiction that dominates american college campuses. it is the belief that america’s most “tolerant,” progressive, and sensitive communities are simultaneously virtual hellholes for marginalized members of the community, justifying emergency   measures designed to end oppression and defend the defenseless. thus, the federal government hypes false statistics that a staggering 20 percent of female students will be victims of sexual assault. thus, campuses implement disciplinary practices and policies that deny due process and treat straight men as guilty until proven innocent. and, colleges claim, it’s all necessary. after all, rape and sexual harassment represent   traumas. false accusations and unjust punishment? well, that’s just a momentary inconvenience  —   a small price to pay for the cause of social justice. tell that to the family of thomas klocke, a student at the university of texas at arlington. the competing factual accounts are simple and difficult to resolve. a gay student accused klocke (who was straight) of typing “gays should die” into the search bar of his browser during a classroom conversation about privilege. when the gay student then typed into his own computer, “i’m gay. ” klocke then allegedly said that the gay student was a “faggot” and that he should consider killing himself. klocke’s account was diametrically opposed. he claimed that the gay student called him “beautiful. ” klocke then typed into his web browser, “stop, i’m straight. ” the gay student replied, “i’m gay” and then allegedly kept glancing at klocke, who eventually got up and moved seats. the dean then allegedly assigned the case to the school’s associate director of academic integrity, who promptly issued an order prohibiting klocke not only from contacting his accuser, he also prohibited him from attending the class where the incident occurred, and  —   crucially  —   from contacting any member of the class, directly or through any other person. later, he reportedly barred klocke’s father, an attorney, from attending a meeting regarding the case, and then “decided” the dispute without following   procedures, without giving klocke the opportunity to contact or call witnesses, and indeed without hearing from any witness who could corroborate either student’s claims. the school, for its part, denies that it departed from mandatory processes and asserts that it “followed its policies and procedures. ” the associate director of academic integrity found klocke responsible for “harassment,” placed him on probation for the remainder of his academic career at the university, and prohibited him from returning to the class where the incident occurred, though he could work on “group projects outside the classroom. ” parents of college boys sometimes take comfort that they can avoid the atmosphere of   hysteria by asking them to follow some relatively simple, commonsense guidelines. don’t drink too much. don’t engage the   culture. use basic manners so that coarse talk isn’t misconstrued. no advice is foolproof, of course, but following those three guidelines  —   in addition to being virtuous on their own merits  —   will help avoid a multitude of problems. not for klocke, however. if his account is true (the university claims it followed proper procedures) then he was doing everything right. he was minding his own business in class, declined an unwelcome sexual advance, and changed seats. if his account is true, a gay student potentially motivated by embarrassment or fear was able to take advantage of the known bias of campus administrators to punish the man that he approached. during my own legal career, i’ve worked with students who’ve experienced many of the same things that thomas klocke’s family claims that he experienced. i’ve represented students whose parents were barred from disciplinary meetings, leaving them to face the wrath of administrators alone. but there’s something else i’ve seen. i’ve seen the cost imposed on students accused of misconduct  —   the fear and the stress as they feel like their reputations, their careers, and their dreams are vanishing before their very eyes. suicide is a complicated phenomenon, and i have no doubt that additional facts will emerge as the case works its way through the judicial system, but no one  —   ever  —   should believe that crackdowns come without cost or that due process is an abstract concept, a mere inconvenience that stands in the way of social justice. one doesn’t minimize sexual assault or sexual harassment by preserving due process. it is not a matter of either protecting the rights of the accuser or respecting the rights of the accused. instead, due process is a basic human right. failing to respect that right inflicts its own unique form of trauma on students, and sometimes that trauma can carry with it horrifying consequences. \n Answer: School Problem, Legal Problem \n Reasoning: The article mentions the suicide of Thomas klocke due to wrongful accusations at his school. \n\n Article: veteran commits suicide in parking lot of long island v.a.    navy veteran peter a. kaisen, of islip, new york, killed himself in the parking lot of the northport veterans affairs medical center on sunday after reportedly being turned away for emergency care. [“kaisen served in the u. s. navy from 1958 to 1962, working on the uss denebola, a ship that delivered refrigerated items and equipment to ships in the fleet,” according to his friend thomas farley, as cited by fox news.   after leaving the navy, kaisen went to work for the long beach police department, where he was severely injured in a car accident, causing him to rely on pain medication over the following decades. the new york times quotes two sources connected to the northport va medical center who said kaisen came there on sunday seeking   care for a mental health issue but was turned away.  according to these sources, the e. r. staff did not tell kaisen that the mental health facility was open 24 hours a day, and a psychologist was on call. for what it’s worth, the long island press notes the medical center’s website claims, “there is always a caring mental health doctor available   in our emergency room. ” he then went to the parking lot outside the medical center’s nursing home facility, where newsday says he was found “bleeding and unresponsive” after shooting himself. although the hospital is currently denying that kaisen presented himself to the emergency room, the new york times’s anonymous sources were scathing in their criticism. “he went to the e. r. and was denied service. and then he went to his car and shot himself,” said one source, identified as a hospital employee. “someone dropped the ball. they should not have turned him away. ” the times reports this is yet another va facility with a troubled history: the northport hospital has been under scrutiny for mismanagement and poor care since the new york times reported in may that it had closed all five of its operating rooms for months after   black particles began falling from air ducts. the office of representative lee zeldin, a republican whose first congressional district is on long island, had been looking into the allegations, eric amidon, mr. zeldin’s chief of staff, said in an email to the times in june. newsday reports that u. s. house representatives steve israel and peter king sent a letter to fbi director james comey on thursday demanding an “expeditious and transparent investigation” of the incident. the fbi was already involved in the case because kaisen’s death occurred on federal property, but there was no suspicion of criminal activity, and kaisen is believed to have been alone when he died. “i’m a vietnam vet  —   disabled from agent orange  —   and he was always looking out for me. he was such a faithful guy. he was such a big advocate for veterans and that’s what makes it’s so sad,” kaisen’s friend thomas farley told fox news. kaisen’s work for his fellow veterans might not be finished yet. “your death is not in vain. through your tragedy, may the bureaucrats change policies to help others that were in your situation,” read one message left on the funeral home’s website, quoted by the long island press. \n Answer: Health Problem \n Reasoning: The article talks about peter a. kaisen who committed suicide by shooting himself due to denial of mental health care at the local veteran care facility. \n\n Article: when should hacking be legal?  when aaron swartz, a prominent programmer and digital activist, was arrested in 2011, he was halfway through a fellowship at harvard’s center for ethics. police took him in after he entered a computer closet at mit and downloaded enormous amounts of data from jstor’s extensive database of academic research. swartz was charged with 13 felonies, 11 of them based on the computer fraud and abuse act, or cfaa. some of the charges hinged on violations of jstor’s terms of service, which prohibit bulk, automated downloads that prevent other customers from accessing its documents. faced with the possibility of decades in prison and up to a million dollars in fines, swartz took his own life in 2013. the ordeal brought cfaa —  a      law that has been updated half a dozen times to keep up with new technology —  under intense scrutiny.   activists decried the law for its vague definitions of “unauthorized access” to a computer, and for punishing researchers and academics who analyze and pick apart computer systems they don’t own or operate. some members of congress tried (and then tried again) to roll back elements of the law, including those that have been used to criminalize    violations. but a bill sponsored by representative zoe lofgren and senator ron wyden —  a pair of   democrats —  died in 2013, and is currently stalled after being reintroduced in 2015. the lawmakers named their bill “aaron’s law” after swartz. now, a group of academic researchers and journalists is suing the government, challenging the constitutionality of part of cfaa. with the help of the american civil liberties union, they’re targeting the portion of the law that makes it illegal to break private companies’ terms of service, claiming that the rule violates researchers’ and journalists’ rights to conduct research and investigations in the public interest, as guaranteed by the first amendment. they come in the form of the seemingly endless blocks of lawyerly fine print that pop up as you sign up for a new service —  the ones nearly every user clicks through without reading. but although they’re an individual’s agreement with a company, cfaa makes violating them a federal crime. one pair of researchers —  christian sandvig, of the university of michigan, and karrie karahalios, of the university of illinois —  is examining   websites, and the other pair —  alan mislove and christo wilson of northeastern university —  is looking at hiring websites. both projects are trying to determine whether the sites serve different search results, listings, and ads to users that appear to be from minority groups. to do so, they’re creating an army of fake profiles and tweaking them to look like they belong to a diverse set of people. but using that tactic —  one that’s very popular among researchers —  could make the professors felons: the terms of most online services, including the largest employment and   websites, prohibit creating multiple profiles, falsifying profile information, and scraping publicly available information with automated scripts. the lawsuit argues that testing for biases is essential to enforcing the fair housing act, which prohibits discrimination in home rentals or sales, and the civil rights act, which bans discrimination in hiring. in the suit, aclu lawyers point out that the government has conducted nationwide undercover testing for illegal biases in housing since the 1970s, and that courts have repeatedly upheld the rights of   testers. the researchers want to know if a user whose profile indicates she’s black will be served different apartments or jobs than one whose profile indicates he’s white. as long as attempts to reform cfaa in congress are stalled, challenging the law in court may be the only way to move the needle. if the case sees any success, it could even breathe life back into aaron’s law, or a similar bill. but until the law is changed, researchers who break companies’ terms of service for work have to keep looking over their shoulders for   corporate lawyers —  or federal prosecutors. \n Answer: Legal Problem \n Reasoning: The article talks about the suicide of Aaron Swartz, whi killed himself after being charged with computer fraud and abuse and facing a hefty penalty of millions and a jail setence spanning decades. \n\n Article: was a va. firefighter humiliated by co-workers online before she killed herself?  the trolls were horrid to her while she was alive. and they continued to be awful after her death. fairfax county firefighter nicole mittendorff, 31, killed herself in virginia’s shenandoah national park, the state medical examiner concluded. but even after the search for her was over, her body was identified and memorial candles began to burn, the cyberbullies  —   who claimed they were her fellow firefighters  —   kept scorching away at mittendorff online.  petula is a columnist for the washington post’s local team who writes about homeless shelters, gun control, high heels, high school choirs, the politics of parenting, jails, abortion clinics, mayors, modern families, strip clubs and gas prices, among other things. view archive   facebook google+ rss   if these trolls are actually members of her firehouse family, then mittendorff becomes another example of a new form of workplace harassment. instead of happening in the office, it happens publicly online.  there is an investigation at mittendorff’s firehouse to find out who posted the vicious online attacks and whether they played a role in her suicide.  “we at fairfax fire and rescue are aware of the posts and are looking into the matter. i assure you that my department can not and will not tolerate bullying of any kind,” fairfax county fire chief richard bowers wrote in a public statement saturday. “we will thoroughly investigate this matter and take any appropriate actions needed. ”   fairfax county fire chief richard bowers said he has ”zero tolerance” for bullying after online postings raise questions about whether firefighter nicole mittendorff was bullied before her death. mittendorf’s body was found in virginia’s shenandoah national park last week. (wusa 9)    [searchers find body thought to be missing fairfax firefighter]   mittendorff’s case offers a chilling window into the persistent harassment women encounter on a daily basis online and at work. in fact, those two forms of ugliness appear to be merging. it means sexual harassment isn’t just   bosses pawing at a secretary or men disparaging female   at the water cooler. it can happen from a home computer and devastate a woman’s reputation and career. “it seems to be a newer thing, doing this online,” said angela hughes, a baltimore county fire captain who is also president of the international association of women in fire and emergency medical services. “cyberbullying on   outlets is a new form of harassment. ” her group gets frequent requests for help from firefighters who believe they are being harassed, threatened or mistreated because of their gender. but recently, the group has seen more online bullying, including some facebook pages that actively harass female firefighters. the problem, she said, is that a firehouse fosters a culture of toughness, of not needing help. and too often, harassment goes on because it’s against the culture to ask for help. online harassment gets directed at   women on social media and by online commenters all the time. the women of wgn, a local television station in chicago, recently took on their abusers when they read some of the nastiest emails, tweets and facebook posts  —   attacks on their weight, their voices, their faces. “keep shoving food down that   of yours,” morning anchor robin baumgarten said, quoting one message. [broadcasters read their rudest comments online]  and we’ve seen it in gamergate, where women in the   world are continually blasted online. that scandal started with feminist   reviewer anita sarkeesian, who had to cancel a university speaking engagement two years ago because one of her many persistent and vulgar online harassers threatened a mass shooting if she spoke. and recently it included zoe quinn, a   developer who was smeared online with death threats and lurid details about her sex life by other gamers and an   who didn’t like her game. from a to z, they get hazed. this brand of workplace harassment operates outside the world of those surveys, workshops and seminars that company lawyers make everyone take, which do nothing to reduce sexism, but only exist to thwart lawsuits in case a caveman boss demands sex for a promotion. even if the trolls ripping mittendorff apart online didn’t work with her, those posts were up there to shame her for her career choice and belittle her within her career. this subversive yet simultaneously very public sexual harassment is becoming increasingly common. we know women still face    harassment in   fields: the military, law enforcement, science, the tech sector. even the women who work for the national park service have offered horror stories. and it’s especially prevalent in firefighting, where i found case after case  —   in rhode island, arizona, utah, florida  —   of women winning sexual harassment cases against their departments in just the past couple of years. in federal reports, congressional testimony, courtroom testimony, personal essays and formal complaints, we hear the same story. women continue to be harassed, belittled, passed over and manipulated by their male   or bosses. \n Answer: Financial/Job Problem \n Reasoning: The article talks about a female firefighter who killed herself after being harassed and bullied by her co-workers online. \n\n Now, read the following news article answer the following 2 questions: \n 1. Is there mention of a person or multiple people who died after committing suicide or killed themselves? Provide your answer in the following format: \n Answer 1: Yes or No  \n 2. Identify which of the above categories does the article belong to. An article can belong to more than one category. Also provide a reasoning for the same in the following format:\n Answer 2: \n Reasoning: \n\n Article: "

In [2]:
import pandas as pd
# below is the datatset annotated by Jay
data = pd.read_csv('final_dataset_06102022 2.csv')

pattern = r'^news[0-6]_'
datacopy = data.copy()
datacopy['Document name'] = data['Document name'].str.replace(pattern, '', regex=True)

relevant = data[data['Yes']==1]
data = data[data['Yes'] == 1]
print(relevant.shape)
print(data.shape)
selected_rows = data[(data[FinancialLabel] > 0) | (data[Legallabel] > 0) | (data[HealthLabel] > 0) | (data[SchoolLabel] > 0)]
financial_problem = data[data[FinancialLabel] > 0]
school_problem = data[data[SchoolLabel] > 0]
legal_problem = data[data[Legallabel] > 0]
health_problem = data[data[HealthLabel] > 0]
unspecified_circumstance = data[data['Unspecified Circumstance'] > 0]

print("Legal Problem : ", legal_problem.shape[0])
print("Financial/Job Problem : ", financial_problem.shape[0])
print("Lack of Access to Health/Mental Health Care : ", health_problem.shape[0])
print("School or Academic Related Problem : ", school_problem.shape[0])
# print("")

(200, 96)
(200, 96)
Legal Problem :  39
Financial/Job Problem :  13
Lack of Access to Health/Mental Health Care :  7
School or Academic Related Problem :  5


In [97]:
# maindf = pd.DataFrame(selected_rows, columns=['Document group', 'Document name', FinancialLabel, SchoolLabel, Legallabel, HealthLabel])
# arrayoftexts = []
# for documentname in maindf['Document name']:
#     with open("articles/"+ documentname + ".txt") as f:
#         text = f.readlines()
#         arrayoftexts.append(text[0])
# maindf['Text'] = arrayoftexts
# maindf = maindf[['Text', FinancialLabel, Legallabel, HealthLabel, SchoolLabel]]

In [98]:
# maindf = pd.read_csv('ben200_gpt4.csv')
# arts = maindf['content'].to_list()
# cat = []
# f = [0] * len(arts)
# h = [0] * len(arts)
# l = [0] * len(arts)
# s = [0] * len(arts)
# #arts = financial_problem_df['Text'].to_list()
# financialpred = [0] * len(arts) 
# reason = [""] * len(arts)

In [99]:
import openai
print(openai.__version__)

0.28.0


In [100]:
# for i in tqdm(range(0, 50)):
#     try:
#         # generate response
#         if (reason[i] != ""):
#             continue
#         print(i)
#         prompt = combinedzeroshot + "\n" + "Article: " + arts[i] + "\n"
#         response = openai.ChatCompletion.create(
#             model="gpt-4",
#             engine="StructuralFactorsOfSuicidegpt4", # engine with high TPM rate limit; no content filter
#             messages=[
#                 {
#                     "role": "user",
#                     "content": prompt,
#                 },
#             ],
#             temperature=0.0
#         )
#         chatgpt_output = json.dumps(response['choices'][0]['message']['content'])
#         print(chatgpt_output)
#         if ("1: No" in chatgpt_output):
#             reason[i] = "NotSuicide"
#             continue
#         if ('Legal Problem' in chatgpt_output):
#             l[i] = 1
#         if ('Financial/Job Problem' in chatgpt_output):
#             f[i] = 1
#         if ('School Problem' in chatgpt_output):
#             s[i] = 1
#         if ('Health Problem' in chatgpt_output):
#             h[i] = 1
#         reason[i] = chatgpt_output
#     except Exception as e:
#         print(i)
#         print(str(e))
        
#     time.sleep(2)
        
# # maindf['Financial - Predicted'] = financialpred
# # maindf['Financial - Reason'] = reason

In [101]:
# maindf['Reasoning'] = reason
# maindf[FinancialPredicted] = f
# maindf[SchoolPredicted] = s
# maindf[HealthPredicted] = h
# maindf[LegalPredicted] = l

In [102]:
# maindf[FinancialLabel] = maindf[FinancialLabel].apply(lambda x: 1 if x > 0 else x)
# maindf[HealthLabel] = maindf[HealthLabel].apply(lambda x: 1 if x > 0 else x)
# maindf[SchoolLabel] = maindf[SchoolLabel].apply(lambda x: 1 if x > 0 else x)
# maindf[Legallabel] = maindf[Legallabel].apply(lambda x: 1 if x > 0 else x)

# financialmatch = len(maindf[(maindf[FinancialPredicted] > 0) & (maindf[FinancialLabel] > 0)])
# legalmatch = len(maindf[(maindf[LegalPredicted] > 0) & (maindf[Legallabel] > 0)])
# schoolmatch = len(maindf[(maindf[SchoolPredicted] > 0) & (maindf[SchoolLabel] > 0)])
# healthmatch = len(maindf[(maindf[HealthPredicted] > 0) & (maindf[HealthLabel] > 0)])

# print(financialmatch)
# print(legalmatch)
# print(schoolmatch)
# print(healthmatch)

# financialmatch = len(maindf[(maindf[FinancialPredicted] > 0) & (maindf[FinancialLabel] == 0)])
# legalmatch = len(maindf[(maindf[LegalPredicted] > 0) & (maindf[Legallabel] == 0)])
# schoolmatch = len(maindf[(maindf[SchoolPredicted] > 0) & (maindf[SchoolLabel] == 0)])
# healthmatch = len(maindf[(maindf[HealthPredicted] > 0) & (maindf[HealthLabel] == 0)])

# print(financialmatch)
# print(legalmatch)
# print(schoolmatch)
# print(healthmatch)

In [103]:
# from sklearn.metrics import hamming_loss, accuracy_score, f1_score
# y_true = maindf[[FinancialLabel, Legallabel, SchoolLabel, HealthLabel]].values
# y_pred = maindf[[FinancialPredicted, LegalPredicted, SchoolPredicted, HealthPredicted]].values
# print(y_true.shape)
# print(y_pred.shape)
# hl = hamming_loss(y_true, y_pred)
# print("Hamming Loss:", hl)

# accuracy = accuracy_score(y_true, y_pred)
# print("Subset accuracy: ", accuracy)

# f1score = f1_score(y_true, y_pred, average='macro')
# print("Macro averaged F1 score: ", f1score)

# f1score = f1_score(y_true, y_pred, average='micro')
# print("Micro averaged F1 score: ", f1score)

# y_true = maindf[FinancialLabel].values
# y_pred = maindf[FinancialPredicted].values
# f1score = f1_score(y_true, y_pred)
# print("Financial problem F1 score: ", f1score)

# y_true = maindf[Legallabel].values
# y_pred = maindf[LegalPredicted].values
# f1score = f1_score(y_true, y_pred)
# print("Legal problem F1 score: ", f1score)

# y_true = maindf[SchoolLabel].values
# y_pred = maindf[SchoolPredicted].values
# f1score = f1_score(y_true, y_pred)
# print("School problem F1 score: ", f1score)

# y_true = maindf[HealthLabel].values
# y_pred = maindf[HealthPredicted].values
# f1score = f1_score(y_true, y_pred)
# print("Health problem F1 score: ", f1score)

In [104]:
# Dataset from ben Horne
# bendata = pd.read_csv('Transfer/article_matches.csv')
# bendata.head(3)
# print(bendata.shape)

# print(bendata[bendata['content'].isna()])

# bendata = bendata[~bendata['content'].isna()]
# suicide_Articles_from_ben_data = bendata['content'].tolist()

# wordcount = 0.0
# maxlen = 0
# minlen = 100000
# maxlens = []
# for art in suicide_Articles_from_ben_data:
#     wordcount += len(art.split())
#     maxlens.append(len(art.split()))
#     maxlen = max(maxlen, len(art.split()))
#     minlen = min(minlen, len(art.split()))

# print("Average word count: ", wordcount/len(suicide_Articles_from_ben_data))
# print("Max word count: ", maxlen)
# print("Min word count: ", minlen)

# ids = bendata['id']
# publisher = []
# date = []
# title = []

# for i in ids.tolist():
#     if (i == ""):
#         print("gadbad1")
#     diffs = i.split("--", maxsplit=2)
#     if (len(diffs) != 3):
#         print("gadbad2")
#         print(i)
#     publisher.append(diffs[0])
#     date.append(diffs[1])
#     title.append(diffs[2])

# bendata["Publisher"] = publisher
# bendata["Date Published"] = date

In [105]:
maindf = pd.read_csv('final_dataset_06102022 2.csv')
print(maindf.shape)

(846, 96)


In [106]:
maindf = maindf[maindf['Yes'] == 1]
financial_problem = maindf[maindf[FinancialLabel] > 0]
school_problem = maindf[maindf[SchoolLabel] > 0]
legal_problem = maindf[maindf[Legallabel] > 0]
health_problem = maindf[maindf[HealthLabel] > 0]
unspecified_circumstance = maindf[maindf['Unspecified Circumstance'] > 0]

print("Legal Problem : ", legal_problem.shape[0])
print("Financial/Job Problem : ", financial_problem.shape[0])
print("Lack of Access to Health/Mental Health Care : ", health_problem.shape[0])
print("School or Academic Related Problem : ", school_problem.shape[0])

Legal Problem :  39
Financial/Job Problem :  13
Lack of Access to Health/Mental Health Care :  7
School or Academic Related Problem :  5


In [107]:
arrayoftexts = []
wordcount = 0.0
for documentname in maindf['Document name']:
    with open("articles/"+ documentname + ".txt") as f:
        text = f.readlines()
        arrayoftexts.append(text[0])
maindf['content'] = arrayoftexts

In [108]:
len(arrayoftexts)

200

In [109]:
# from openai import OpenAI
# openai = OpenAI(
#     api_key="dUjtO4X6TkCXXHyTfeZcujA9VfXaJrrw",
#     base_url="https://api.deepinfra.com/v1/openai",
# )



In [110]:
arts = maindf['content'].to_list()
f = [0] * len(arts)
h = [0] * len(arts)
l = [0] * len(arts)
s = [0] * len(arts)
n = [0] * len(arts)
reason = [""] * len(arts)
erroredout = [0] * len(arts)

In [88]:
test = ""
for i in tqdm(range(0, len(arts))):
    try:
        prompt = combinedfewshot + arts[i]
        #response = openai.ChatCompletion.create(
        # response = openai.chat.completions.create(
        #     model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        #     #engine="StructuralFactorsOfSuicideturbo16k", # engine with high TPM rate limit; no content filter
        #     messages=[
        #         {
        #             "role": "user",
        #             "content": prompt,
        #         },
        #     ],
        #     temperature=0.0
        # )
        # chatgpt_output = response.choices[0].message.content
        response = openai.ChatCompletion.create(
            model="gpt-4",
            engine="StructuralFactorsOfSuicidegpt4", # engine with high TPM rate limit; no content filter
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                },
            ],
            temperature=0.0
        )
        chatgpt_output = json.dumps(response['choices'][0]['message']['content'])
        print(chatgpt_output)
        test = chatgpt_output
        if ("1: No" in chatgpt_output):
            reason[i] = "NotSuicide"
            continue
        split = chatgpt_output.split('\\n')
        for sp in split:
            if "Answer 2" in sp:
                chatgpt_output = sp
        #print(chatgpt_output)
        n[i] = 1
        if ('Legal Problem' in chatgpt_output):
            l[i] = 1
            n[i] = 0
        if ('Financial/Job Problem' in chatgpt_output):
            f[i] = 1
            n[i] = 0
        if ('School Problem' in chatgpt_output):
            s[i] = 1
            n[i] = 0
        if ('Health Problem' in chatgpt_output):
            h[i] = 1
            n[i] = 0
        reason[i] = test
        # gt = ""
        # if (maindf.iloc[i][FinancialLabel] > 0):
        #     gt += "Financial "
        # if (maindf.iloc[i][Legallabel] > 0):
        #     gt += "Legal "
        # if (maindf.iloc[i][HealthLabel] > 0):
        #     gt += "Health "
        # if (maindf.iloc[i][SchoolLabel] > 0):
        #     gt += "School "
        # print("ground truth: ", gt)
    except Exception as e:
        print(i)
        if ("reduce the length of the messages." in str(e)):
            print("TooManyTokens")
            reason[i] = "TooManyTokens"
        else:
            print(str(e))
        #print(str(e))
        erroredout[i] = 1
        
        

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 1/200 [00:12<42:39, 12.86s/it]

"Answer 1: Yes\nAnswer 2: None\nReasoning: The article talks about the suicide of Rashaan Salaam, a former football player. However, the reasons for his suicide are not clearly mentioned in the article. It is mentioned that he was dealing with mental health problems and was a recluse, but it is not clear whether these issues were due to financial/job problems, school problems, health problems, or legal problems. Therefore, the article does not fit into any of the given categories."


  1%|          | 2/200 [00:20<32:42,  9.91s/it]

"Answer 1: No\nAnswer 2: None\nReasoning: The article does not mention any suicide or any person who killed themselves. It talks about the death of King Bhumibol Adulyadej of Thailand due to natural causes."


...

"Answer 1: Yes\nAnswer 2: Legal Problem\nReasoning: The article talks about a police officer who killed himself and left a suicide note stating his involvement in an underage prostitution scheme. The note also led to a criminal investigation of several other officers."


100%|█████████▉| 199/200 [27:22<00:08,  8.84s/it]

"Answer 1: Yes\nAnswer 2: None\nReasoning: The article talks about a 12-year-old girl who committed suicide and live streamed it. However, the article does not provide any specific reason related to financial/job problems, school problems, health problems or legal problems that led to her suicide. The article mentions that she was sexually abused by a family member, but this does not fall into any of the provided categories."


100%|██████████| 200/200 [27:29<00:00,  8.25s/it]

"Answer 1: Yes\nAnswer 2: Financial/Job Problem\nReasoning: The article talks about Nicole Mittendorff, a firefighter who killed herself after being bullied and harassed by her co-workers online."


In [90]:
maindf[FinancialPredicted] = f
maindf[SchoolPredicted] = s
maindf[HealthPredicted] = h
maindf[LegalPredicted] = l
maindf['None - Predicted'] = n
maindf['Reasoning'] = reason
maindf['Error'] = erroredout


In [ ]:
maindf.to_csv('kaggle200_gpt48kfewshotresults.csv')

In [9]:
maindf = pd.read_csv('/Users/minijain/Downloads/SocWebLab/Structural_Violence/kaggle200_gpt48kzeroshotresultsmodified.csv')

In [10]:
f = maindf[FinancialPredicted]
s = maindf[SchoolPredicted]
h = maindf[HealthPredicted]
l = maindf[LegalPredicted] 
n = maindf['None - Predicted']

In [11]:
actualf = maindf[FinancialLabel].to_list()
actuall = maindf[Legallabel].to_list()
actuals = maindf[SchoolLabel].to_list()
actualh = maindf[HealthLabel].to_list()

actualf = [1 if x > 0 else x for x in actualf]
actuall = [1 if x > 0 else x for x in actuall]
actuals = [1 if x > 0 else x for x in actuals]
actualh = [1 if x > 0 else x for x in actualh]

from sklearn.metrics import recall_score, f1_score, accuracy_score, precision_score, roc_auc_score

print("Financial Accuracy: ", accuracy_score(actualf, f))
print("Financial Precision: ", precision_score(actualf, f))
print("Financial Recall: ", recall_score(actualf, f))
print("Financial F1 score: ", f1_score(actualf, f))
print("Financial AUC score: ", roc_auc_score(actualf, f))

print("Legal Accuracy: ", accuracy_score(actuall, l))
print("Legal Precision: ", precision_score(actuall, l))
print("Legal Recall: ", recall_score(actuall, l))
print("Legal F1 score: ", f1_score(actuall, l))
print("Legal AUC score: ", roc_auc_score(actuall, l))

print("School Accuracy: ", accuracy_score(actuals, s))
print("School Precision: ", precision_score(actuals, s))
print("School Recall: ", recall_score(actuals, s))
print("School F1 score: ", f1_score(actuals, s))
print("School AUC score: ", roc_auc_score(actuals, s))

print("Health Accuracy: ", accuracy_score(actualh, h))
print("Health Precision: ", precision_score(actualh, h))
print("Health Recall: ", recall_score(actualh, h))
print("Health F1 score: ", f1_score(actualh, h))
print("Health AUC score: ", roc_auc_score(actualh, h))


Financial Accuracy:  0.93
Financial Precision:  0.4444444444444444
Financial Recall:  0.3076923076923077
Financial F1 score:  0.3636363636363637
Financial AUC score:  0.6404771698889345
Legal Accuracy:  0.85
Legal Precision:  0.5737704918032787
Legal Recall:  0.8974358974358975
Legal F1 score:  0.7
Legal AUC score:  0.8679726071030419
School Accuracy:  0.945
School Precision:  0.3125
School Recall:  1.0
School F1 score:  0.47619047619047616
School AUC score:  0.9717948717948718
Health Accuracy:  0.72
Health Precision:  0.09836065573770492
Health Recall:  0.8571428571428571
Health F1 score:  0.17647058823529413
Health AUC score:  0.7860843819393042


In [12]:
from sklearn.metrics import hamming_loss, accuracy_score, f1_score

def make_positive_one(x):
    return 1 if x > 0 else x

# Apply the function to the DataFrame column
maindf[FinancialLabel] = maindf[FinancialLabel].apply(make_positive_one)
maindf[SchoolLabel] = maindf[SchoolLabel].apply(make_positive_one)
maindf[Legallabel] = maindf[Legallabel].apply(make_positive_one)
maindf[HealthLabel] = maindf[HealthLabel].apply(make_positive_one)

y_true = maindf[[FinancialLabel, Legallabel, SchoolLabel, HealthLabel]].values
y_pred = maindf[[FinancialPredicted, LegalPredicted, SchoolPredicted, HealthPredicted]].values
print(y_true.shape)
print(y_pred.shape)
hl = hamming_loss(y_true, y_pred)
print("Hamming Loss:", hl)

accuracy = accuracy_score(y_true, y_pred)
print("Subset accuracy: ", accuracy)

f1score = f1_score(y_true, y_pred, average='macro')
print("Macro averaged F1 score: ", f1score)

f1score = f1_score(y_true, y_pred, average='micro')
print("Micro averaged F1 score: ", f1score)

y_true = maindf[FinancialLabel].values
y_pred = maindf[FinancialPredicted].values
f1score = f1_score(y_true, y_pred)
print("Financial problem F1 score: ", f1score)

y_true = maindf[Legallabel].values
y_pred = maindf[LegalPredicted].values
f1score = f1_score(y_true, y_pred)
print("Legal problem F1 score: ", f1score)

y_true = maindf[SchoolLabel].values
y_pred = maindf[SchoolPredicted].values
f1score = f1_score(y_true, y_pred)
print("School problem F1 score: ", f1score)

y_true = maindf[HealthLabel].values
y_pred = maindf[HealthPredicted].values
f1score = f1_score(y_true, y_pred)
print("Health problem F1 score: ", f1score)

(200, 4)
(200, 4)
Hamming Loss: 0.13875
Subset accuracy:  0.515
Macro averaged F1 score:  0.4290743570155335
Micro averaged F1 score:  0.47393364928909953
Financial problem F1 score:  0.3636363636363637
Legal problem F1 score:  0.7
School problem F1 score:  0.47619047619047616
Health problem F1 score:  0.17647058823529413


In [ ]:
# import matplotlib.pyplot as plt

# # Plotting the array
# bin_edges = [0, 2000, 4000, 6000, 8000, 10000, 20000]  # Adjust these edges according to your needs

# # Plotting the histogram
# plt.hist(maxlens, bins=bin_edges, edgecolor='black')

# # Adding labels and title
# plt.xlabel('Length of Article')
# plt.ylabel('Frequency')
# plt.title('Length Distribution')

# # Displaying the plot
# plt.show()

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('finalbendatasetwithpreds.csv')

arts = data['Reasoning'].to_list()
f = [0] * len(arts)
h = [0] * len(arts)
l = [0] * len(arts)
s = [0] * len(arts)
n = [0] * len(arts)
#arts = financial_problem_df['Text'].to_list()
financialpred = [0] * len(arts) 
reason = [""] * len(arts)

In [ ]:
multi = 0
fi = 0
sc = 0
le = 0
he = 0

finalreason = [""] * len(arts)

for i in range(len(arts)):
    chatgpt_output = arts[i]
    if ("1: No" in chatgpt_output):
        reason[i] = "NotSuicide"
        continue
    if ("NotSuicide" in chatgpt_output or "TooManyTokens" in chatgpt_output):
        continue
    split = chatgpt_output.split('\\n')
    for sp in split:
        if "Answer 2" in sp:
            chatgpt_output = sp
        if ("Reasoning" in sp):
            finalreason[i] = sp.replace("Reasoning: ", "")
    #print(chatgpt_output)
    n[i] = 1
    if ('Legal Problem' in chatgpt_output):
        l[i] = 1
        n[i] = 0
    if ('Financial/Job Problem' in chatgpt_output):
        f[i] = 1
        n[i] = 0
    if ('School Problem' in chatgpt_output):
        s[i] = 1
        n[i] = 0
    if ('Health Problem' in chatgpt_output):
        h[i] = 1
        n[i] = 0
    # if (l[i] + f[i] + h[i] + s[i] > 1):
    #     multi = multi + 1
    #     if (f[i] == 1):
    #         fi = fi + 1 
    #     if (h[i] == 1):
    #         he = he + 1 
    #     if (l[i] == 1):
    #         le = le + 1 
    #     if (s[i] == 1):
    #         sc = sc + 1 
        #print(chatgpt_output)
#print(multi)

#print(fi)
#print(le)
#print(sc)
#print(he)

#print("Multi label: ",data[(data[HealthPredicted] + data[FinancialPredicted] + data[LegalPredicted] + data[SchoolPredicted] > 1)].shape[0])

#print("Multi label: ",data[(data[HealthPredicted] + data[FinancialPredicted] + data[LegalPredicted] + data[SchoolPredicted] > 1)].shape[0])


In [8]:
import pandas as pd
import numpy as np
data = pd.read_csv('/Users/minijain/Downloads/SocWebLab/Structural_Violence/kaggle200_gpt48kzeroshotresults.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/minijain/Downloads/SocWebLab/Structural_Violence/kaggle200_gpt432kzeroshotresults.csv'

In [7]:
import pandas as pd
# below is the datatset annotated by Jay
#data = pd.read_csv('final_dataset_06102022 2.csv')

pattern = r'^news[0-6]_'
datacopy = data.copy()
datacopy['Document name'] = data['Document name'].str.replace(pattern, '', regex=True)

relevant = data[data['Yes']==1]
data = data[data['Yes'] == 1]
print(relevant.shape)
print(data.shape)
selected_rows = data[(data[FinancialLabel] > 0) | (data[Legallabel] > 0) | (data[HealthLabel] > 0) | (data[SchoolLabel] > 0)]
financial_problem = data[data[FinancialLabel] > 0]
school_problem = data[data[SchoolLabel] > 0]
legal_problem = data[data[Legallabel] > 0]
health_problem = data[data[HealthLabel] > 0]
unspecified_circumstance = data[data['Unspecified Circumstance'] > 0]

print("Financial/Job Problem : ", financial_problem.shape[0])
print("Legal Problem : ", legal_problem.shape[0])
print("School or Academic Related Problem : ", school_problem.shape[0])
print("Lack of Access to Health/Mental Health Care : ", health_problem.shape[0])

# print("")

(200, 105)
(200, 105)
Financial/Job Problem :  13
Legal Problem :  39
School or Academic Related Problem :  5
Lack of Access to Health/Mental Health Care :  7


In [14]:
data = maindf
print(data[data[FinancialPredicted] > 0].shape[0])
print(data[data[LegalPredicted] > 0].shape[0])
print(data[data[SchoolPredicted] > 0].shape[0])
print(data[data[HealthPredicted] > 0].shape[0])

9
61
16
61


In [21]:
legal = data[data[LegalPredicted] > 0]
legal = legal[data[Legallabel] <= 0]
print(len(legal['Reasoning'].to_list()))


26


/var/folders/wv/r6mcv43147qf407r3r774phc0000gn/T/ipykernel_29306/453722469.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  legal = legal[data[Legallabel] <= 0]
